In [0]:
%pip install -r requirements.lock

  Using cached adlfs-2025.8.0-py3-none-any.whl.metadata (7.7 kB)
  Using cached aiohappyeyeballs-2.6.1-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiohttp-3.13.2-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.manylinux_2_28_x86_64.whl.metadata (8.1 kB)
  Using cached aiosignal-1.4.0-py3-none-any.whl.metadata (3.7 kB)
  Using cached attrs-25.4.0-py3-none-any.whl.metadata (10 kB)
  Using cached azure_core-1.36.0-py3-none-any.whl.metadata (47 kB)
  Using cached azure_datalake_store-0.0.53-py2.py3-none-any.whl.metadata (19 kB)
  Using cached azure_identity-1.25.1-py3-none-any.whl.metadata (88 kB)
  Using cached azure_storage_blob-12.27.1-py3-none-any.whl.metadata (26 kB)
  Using cached bokeh-3.8.1-py3-none-any.whl.metadata (10 kB)
  Using cached certifi-2025.11.12-py3-none-any.whl.metadata (2.5 kB)
  Using cached cffi-2.0.0-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (2.6 kB)
  Using cached cftime-1.6.5-cp312-cp312-manylinux2014_x86_64.manylinux_2_17_x

In [0]:
import nest_asyncio
nest_asyncio.apply()

In [0]:
import fsspec
import glob
import json
import os

import xarray as xr

from pathlib import Path

from kerchunk.hdf import SingleHdf5ToZarr
from kerchunk.combine import MultiZarrToZarr

Create Kerchunk references for multiple NetCDF files.

In [0]:
def create_references(nc_pattern, output_dir='references'):
    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)
    
    nc_files = sorted(glob.glob(nc_pattern))
    print(f"Processing {len(nc_files)} NetCDF files")
    
    reference_files = []
    
    # Create individual references
    for i, nc_file in enumerate(nc_files, 1):
        ref_file = output_dir / f"{Path(nc_file).stem}.json"
        
        with fsspec.open(nc_file, mode='rb') as f:
            h5chunks = SingleHdf5ToZarr(f, nc_file, inline_threshold=100)
            refs = h5chunks.translate()
        
        with open(ref_file, 'w') as f:
            json.dump(refs, f)
        
        reference_files.append(str(ref_file))
        
        if i % 10 == 0:
            print(f"  Created {i}/{len(nc_files)} references...")
    
    print(f"  Created {len(reference_files)} reference files")
    
    # Combine references
    print("\nCombining references...")
    mzz = MultiZarrToZarr(
        reference_files,
        concat_dims=['valid_time'],
        identical_dims=['number', 'latitude', 'longitude'],
        coo_map={'valid_time': 'INDEX'}
    )
    
    combined_refs = mzz.translate()
    
    # Save combined reference
    combined_file = output_dir / 'master.json'
    with open(combined_file, 'w') as f:
        json.dump(combined_refs, f)
    
    print(f"Combined reference saved to: {combined_file}")
    
    return combined_file

In [0]:
create_references(
    "/Volumes/stuart/lseg/netcdf/weather/benchmarking/*.nc",
    "/Volumes/stuart/lseg/netcdf/weather/benchmarking/references",
)

Processing 100 NetCDF files
  Created 10/100 references...
  Created 20/100 references...
  Created 30/100 references...
  Created 40/100 references...
  Created 50/100 references...
  Created 60/100 references...
  Created 70/100 references...
  Created 80/100 references...
  Created 90/100 references...
  Created 100/100 references...
  Created 100 reference files

Combining references...
Combined reference saved to: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json


PosixPath('/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json')

In [0]:
%fs ls /Volumes/stuart/lseg/netcdf/weather/benchmarking/references

path,name,size,modificationTime
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/00901de0a9454a0a90d10c581f18a06f.json,00901de0a9454a0a90d10c581f18a06f.json,3974,1764331046000
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/041c265dedba4ad5bcb6fcf2dadfd71b.json,041c265dedba4ad5bcb6fcf2dadfd71b.json,3974,1764331049000
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/07ae542a930647a2a842038b74d40015.json,07ae542a930647a2a842038b74d40015.json,3974,1764331051000
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/07e60d54cbb64228b4c41ad38f4bfc62.json,07e60d54cbb64228b4c41ad38f4bfc62.json,3974,1764331054000
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/0ee1d0fd00da4a99b1cfd167b19cb5f2.json,0ee1d0fd00da4a99b1cfd167b19cb5f2.json,3974,1764331056000
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/103df1fa27e54f9f803b7122a6a213d6.json,103df1fa27e54f9f803b7122a6a213d6.json,3974,1764331059000
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/1434778eaef0452c9dc981274f779abd.json,1434778eaef0452c9dc981274f779abd.json,3974,1764331062000
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/159e98301ec6474fba91dc75a6a4ba64.json,159e98301ec6474fba91dc75a6a4ba64.json,3974,1764331064000
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/16b5d5d0bcfd4415829122f539adbd26.json,16b5d5d0bcfd4415829122f539adbd26.json,3974,1764331067000
dbfs:/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/17a902c908e24b3fa03e0b7877c459b5.json,17a902c908e24b3fa03e0b7877c459b5.json,3974,1764331069000


In [0]:
%sh cat /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json

{"version": 1, "refs": {".zgroup": "{\"zarr_format\":2}", ".zattrs": "{\"GRIB_centre\":\"ecmf\",\"GRIB_centreDescription\":\"European Centre for Medium-Range Weather Forecasts\",\"GRIB_subCentre\":0,\"Conventions\":\"CF-1.7\",\"institution\":\"European Centre for Medium-Range Weather Forecasts\",\"history\":\"2025-11-27T10:58 GRIB to CDM+CF via cfgrib-0.9.15.1\\/ecCodes-2.42.0 with {\\\"source\\\": \\\"tmp4m19ge78\\/data.grib\\\", \\\"filter_by_keys\\\": {\\\"stream\\\": [\\\"enda\\\"], \\\"stepType\\\": [\\\"instant\\\"]}, \\\"encode_cf\\\": [\\\"parameter\\\", \\\"time\\\", \\\"geography\\\", \\\"vertical\\\"]}\"}", "valid_time/.zarray": "{\n  \"shape\": [\n    100\n  ],\n  \"chunks\": [\n    100\n  ],\n  \"dtype\": \"<i8\",\n  \"fill_value\": null,\n  \"order\": \"C\",\n  \"filters\": null,\n  \"dimension_separator\": \".\",\n  \"compressor\": null,\n  \"zarr_format\": 2\n}", "valid_time/.zattrs": "{\n  \"_ARRAY_DIMENSIONS\": [\n    \"valid_time\"\n  ],\n  \"long_name\": \"time\",\n

In [0]:
def open_kerchunk_dataset(reference_file):
    """Open dataset from Kerchunk reference."""
    print(f"\nOpening dataset from reference: {reference_file}")

    with open(reference_file) as f:
        refs = json.load(f)

    fs = fsspec.filesystem(
        "reference",
        fo=refs,
        remote_protocol="local",
    )

    mapper = fs.get_mapper("")

    ds = xr.open_dataset(mapper, engine="zarr", backend_kwargs={"consolidated": False})

    print(f"  Dataset opened successfully!")
    print(f"  Dimensions: {dict(ds.sizes)}")
    print(f"  Variables: {list(ds.data_vars)}")

    return ds

In [0]:
ds = open_kerchunk_dataset("/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json")
ds


Opening dataset from reference: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json
  Dataset opened successfully!
  Dimensions: {'valid_time': 100, 'latitude': 361, 'longitude': 720, 'number': 50}
  Variables: ['t2m']


<xarray.Dataset> Size: 5GB
Dimensions:     (valid_time: 100, latitude: 361, longitude: 720, number: 50)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 800B 2025-11-20T18:00:00 ... 2026...
  * latitude    (latitude) float64 3kB 90.0 89.5 89.0 88.5 ... -89.0 -89.5 -90.0
  * longitude   (longitude) float64 6kB 0.0 0.5 1.0 1.5 ... 358.5 359.0 359.5
  * number      (number) int64 400B 0 1 2 3 4 5 6 7 ... 42 43 44 45 46 47 48 49
Data variables:
    t2m         (number, valid_time, latitude, longitude) float32 5GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-27T10:58 GRIB to CDM+CF via cfgrib-0.9.1...

In [0]:
ds.info()

xarray.Dataset {
dimensions:
	valid_time = 100 ;
	latitude = 361 ;
	longitude = 720 ;
	number = 50 ;

variables:
	datetime64[ns] valid_time(valid_time) ;
		valid_time:long_name = time ;
		valid_time:standard_name = time ;
	float64 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:standard_name = latitude ;
		latitude:long_name = latitude ;
		latitude:stored_direction = decreasing ;
	float64 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:standard_name = longitude ;
		longitude:long_name = longitude ;
	int64 number(number) ;
		number:long_name = ensemble member numerical id ;
		number:units = 1 ;
		number:standard_name = realization ;
	float32 t2m(number, valid_time, latitude, longitude) ;
		t2m:GRIB_paramId = 167 ;
		t2m:GRIB_dataType = an ;
		t2m:GRIB_numberOfPoints = 259920 ;
		t2m:GRIB_typeOfLevel = surface ;
		t2m:GRIB_stepUnits = 1 ;
		t2m:GRIB_stepType = instant ;
		t2m:GRIB_gridType = regular_ll ;
		t2m:GRIB_uvRelativeToGrid = 0 ;
		t2m:GRI

In [0]:
ds.t2m[-1][-1][-1].values

array([258.09497, 269.8234 , 278.78317, 292.10355, 260.3426 , 288.24457,
       231.16801, 241.9859 , 297.5479 , 243.32307, 285.65073, 262.16055,
       306.5065 , 257.3265 , 267.6651 , 281.25192, 313.25488, 257.95456,
       277.07095, 253.82823, 265.35147, 232.75746, 244.94478, 304.6257 ,
       303.8453 , 304.65216, 293.10803, 267.75433, 256.49646, 276.96014,
       301.16568, 311.07062, 263.88986, 318.3675 , 224.51628, 227.3476 ,
       286.44748, 235.21408, 306.64487, 272.29102, 271.37976, 269.70648,
       270.0126 , 311.05768, 268.86087, 258.5799 , 311.6846 , 298.98016,
       232.676  , 266.10165, 223.17569, 233.39348, 299.62283, 304.24155,
       249.06187, 262.67648, 304.25342, 241.27763, 281.44867, 272.3404 ,
       277.0518 , 224.49066, 241.05273, 254.22539, 288.36472, 295.5889 ,
       287.15457, 259.36438, 259.06046, 287.12708, 278.11383, 316.01962,
       241.83145, 242.04077, 224.95616, 311.17538, 255.7322 , 308.4316 ,
       279.04736, 238.98166, 233.4351 , 239.02121, 

In [0]:
dbutils.notebook.exit("0")

## Option
Use Spark to generate the references before collecting and combining.

In [0]:
import pyspark.sql.functions as F
import pyspark.sql.types as ST

In [0]:
def create_references_spark(nc_pattern, output_dir):

    output_dir = Path(output_dir)
    output_dir.mkdir(exist_ok=True)

    root = Path(nc_pattern).parent
    filter_suffix = Path(nc_pattern).suffix

    nc_files_df = (
      spark.createDataFrame(dbutils.fs.ls(f"{root}"))
      .where(F.lower(F.col("path")).like(f"%{filter_suffix.lower()}"))
      .withColumn("path", F.regexp_replace(F.col("path"), "dbfs:/", "/"))
      )

    @F.udf(ST.StringType())
    def h5translate(path):
      with fsspec.open(path, mode='rb') as f:
        h5chunks = SingleHdf5ToZarr(f, path, inline_threshold=100)
        refs = h5chunks.translate()
      return json.dumps(refs)

    refs_df = (
      nc_files_df
      .withColumn("refs", h5translate(F.col("path")))
    )

    reference_dicts = [json.loads(r.refs) for r in (
      refs_df
      .select(F.col("refs"))
      .collect()
    )]

    print(f"  Created {len(reference_dicts)} reference sets")
    
    # Combine references
    print("\nCombining references...")
    mzz = MultiZarrToZarr(
        reference_dicts,
        concat_dims=['valid_time'],
        identical_dims=['number', 'latitude', 'longitude'],
        coo_map={'valid_time': 'INDEX'}
    )
    
    combined_refs = mzz.translate()
    
    # Save combined reference
    combined_file = output_dir / 'master.json'
    with open(combined_file, 'w') as f:
        json.dump(combined_refs, f)
    
    print(f"Combined reference saved to: {combined_file}")
    
    return combined_file

In [0]:
create_references_spark(
    "/Volumes/stuart/lseg/netcdf/weather/benchmarking/*.nc",
    "/Volumes/stuart/lseg/netcdf/weather/benchmarking/references",
)

  Created 100 reference sets

Combining references...
Combined reference saved to: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json


PosixPath('/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json')

In [0]:
ds = open_kerchunk_dataset("/Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json")
ds


Opening dataset from reference: /Volumes/stuart/lseg/netcdf/weather/benchmarking/references/master.json
  Dataset opened successfully!
  Dimensions: {'valid_time': 100, 'latitude': 361, 'longitude': 720, 'number': 50}
  Variables: ['t2m']


<xarray.Dataset> Size: 5GB
Dimensions:     (valid_time: 100, latitude: 361, longitude: 720, number: 50)
Coordinates:
  * valid_time  (valid_time) datetime64[ns] 800B 2025-11-20T18:00:00 ... 2026...
  * latitude    (latitude) float64 3kB 90.0 89.5 89.0 88.5 ... -89.0 -89.5 -90.0
  * longitude   (longitude) float64 6kB 0.0 0.5 1.0 1.5 ... 358.5 359.0 359.5
  * number      (number) int64 400B 0 1 2 3 4 5 6 7 ... 42 43 44 45 46 47 48 49
Data variables:
    t2m         (number, valid_time, latitude, longitude) float32 5GB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-11-27T10:58 GRIB to CDM+CF via cfgrib-0.9.1...

In [0]:
ds.info()

xarray.Dataset {
dimensions:
	valid_time = 100 ;
	latitude = 361 ;
	longitude = 720 ;
	number = 50 ;

variables:
	datetime64[ns] valid_time(valid_time) ;
		valid_time:long_name = time ;
		valid_time:standard_name = time ;
	float64 latitude(latitude) ;
		latitude:units = degrees_north ;
		latitude:standard_name = latitude ;
		latitude:long_name = latitude ;
		latitude:stored_direction = decreasing ;
	float64 longitude(longitude) ;
		longitude:units = degrees_east ;
		longitude:standard_name = longitude ;
		longitude:long_name = longitude ;
	int64 number(number) ;
		number:long_name = ensemble member numerical id ;
		number:units = 1 ;
		number:standard_name = realization ;
	float32 t2m(number, valid_time, latitude, longitude) ;
		t2m:GRIB_paramId = 167 ;
		t2m:GRIB_dataType = an ;
		t2m:GRIB_numberOfPoints = 259920 ;
		t2m:GRIB_typeOfLevel = surface ;
		t2m:GRIB_stepUnits = 1 ;
		t2m:GRIB_stepType = instant ;
		t2m:GRIB_gridType = regular_ll ;
		t2m:GRIB_uvRelativeToGrid = 0 ;
		t2m:GRI

In [0]:
ds.t2m[-1][-1][-1].values

array([258.09497, 269.8234 , 278.78317, 292.10355, 260.3426 , 288.24457,
       231.16801, 241.9859 , 297.5479 , 243.32307, 285.65073, 262.16055,
       306.5065 , 257.3265 , 267.6651 , 281.25192, 313.25488, 257.95456,
       277.07095, 253.82823, 265.35147, 232.75746, 244.94478, 304.6257 ,
       303.8453 , 304.65216, 293.10803, 267.75433, 256.49646, 276.96014,
       301.16568, 311.07062, 263.88986, 318.3675 , 224.51628, 227.3476 ,
       286.44748, 235.21408, 306.64487, 272.29102, 271.37976, 269.70648,
       270.0126 , 311.05768, 268.86087, 258.5799 , 311.6846 , 298.98016,
       232.676  , 266.10165, 223.17569, 233.39348, 299.62283, 304.24155,
       249.06187, 262.67648, 304.25342, 241.27763, 281.44867, 272.3404 ,
       277.0518 , 224.49066, 241.05273, 254.22539, 288.36472, 295.5889 ,
       287.15457, 259.36438, 259.06046, 287.12708, 278.11383, 316.01962,
       241.83145, 242.04077, 224.95616, 311.17538, 255.7322 , 308.4316 ,
       279.04736, 238.98166, 233.4351 , 239.02121, 